<a href="https://colab.research.google.com/github/diogomdbarros/automatiza-o-script/blob/main/script_usuarios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Script de Automação de Texto para Criação de Usuários


## 1. Bibliotecas

### 1.1 Importando Bibliotecas

In [ ]:
import pandas as pd

### 2. Carregando arquivo CSV

In [ ]:
df = pd.read_csv("SOLICITAÇÃO DE ACESSOS.csv")

## 3. Tratamento dos Dados

### 3.1 Removendo Colunas

Remoção de colunas que não serão utilizadas no dataframe para solução do projeto.

In [ ]:
# Exclui colunas não utilizadas do DataFrame
df.drop('MATRICULA', axis=1, inplace=True)
df.drop('DATA DE ADMISSÃO', axis=1, inplace=True)
df.drop('NOME DO SOLICITANTE', axis=1, inplace=True)
df.drop('HORARIO DE EXPEDIENTE(OPERADOR)', axis=1, inplace=True)
df.drop('Unnamed: 10', axis=1, inplace=True)
# Renomeando Colunas
df.columns = ['data', 'nome', 'setor', 'cpf', 'filial', 'telefone', 'email_solicitante', 'supervisor']

In [ ]:
df.head()

### 3.2 Função seletora de usuário espelho

Função criada para adicionar uma coluna ( setor ) ao dataframe, com dados de um usuário já existente ( usuário  espelho ) para auxílio do TI na criação dos novos acessos com as permissõs já definidas.

In [ ]:
# Função para localizar o setor e atribuir o Usuario_Espelho correspondente
def definir_usuario_espelho(setor):
    if setor == 'ADM':
        return 'joao.pedro user/0001'
    elif setor == 'RH':
        return 'maria.flor user/0002'
    elif setor == 'OPERAÇÃO':
        return 'ana.clara user/0003'
    else:
        return 'pedro.silva user/0004'

# Adiciona a coluna usuário espelho conforme o setor localizado na coluna "setor"
df['usuario_espelho'] = df['setor'].apply(definir_usuario_espelho)


### 3.3 Função Formatação de CPF

A função formata a coluna CPF verificando se os mesmos possuem 11 digitos e se todos os caracteres são numéricos.

In [ ]:
def formatar_cpf(cpf):
    # Remove qualquer caractere não numérico
    cpf = ''.join(filter(str.isdigit, cpf))
    # Garantir que o CPF tenha 11 dígitos e aplicar a formatação
    if len(cpf) == 11:
        return f'{cpf[:3]}.{cpf[3:6]}.{cpf[6:9]}-{cpf[9:]}'
    return cpf  # Retorna sem formatação se não tiver 11 dígitos

# Adiciona a formatação a coluna "cpf"
df['cpf'] = df['cpf'].apply(formatar_cpf)

### 3.4 Função Formatação de Telefone

A função formata a coluna ( telefone ) verificando se os mesmos possuem 11 digitos e se todos os caracteres são numéricos.

In [ ]:
def formatar_telefone(telefone):
    # Remove qualquer caractere não númerico
    telefone = ''.join(filter(str.isdigit, telefone))
    # Formatar telefone
    if len(telefone) == 11:  # Celular
        return f'({telefone[:2]}) {telefone[2:7]}-{telefone[7:]}'
    return telefone  # Retorna sem formatação se não tiver 11 dígitos

# Adiciona a formatação a coluna "telefone"
df['telefone'] = df['telefone'].apply(formatar_telefone)

In [ ]:
df.tail(4)

## 4. Texto Base

Aqui a variável "script" guarda o texto a ser gerado segundo as necessidades do processo. Foram adicionados placeholders {} para o preenchimento automático com os dados do dataframe.

In [ ]:
script = """
Criar AD - {nome}

Boa dia!
Poderiam realizar criação do AD {nome}, por favor?
Cargo: Teleoperador
Gerente: {gerente}
Desde já agradeço

Criar usuário - {nome}

Boa dia!
Poderiam realizar criação no SIS/ADM/QUEUE para {nome}, por favor?
Setor: {setor}
Cargo: Teleoperador
CPF: {cpf}
Gerente: {gerente}
Usuario espelho: {usuario_espelho}
Local de trabalho: {filial}
Desde já agradeço

Telefone: {telefone}

"""

# 5. Gerando Texto por Intervalo

## 5.1. O objetivo aqui é selecionar um intervalo de linhas do DataFrame df.



In [ ]:
# Selecionar linhas
df_selecionado = df.loc[df.index[466:470]]

# Gerar textos apenas para essas linhas
for index, row in df_selecionado.iterrows():
    texto_personalizado = script.format(
        nome=row['nome'],
        cpf=row['cpf'],
        setor=row['setor'],
        filial=row['filial'],
        usuario_espelho=row['usuario_espelho'],
        telefone=row['telefone']
    )
    print(f"Texto gerado para {row['nome']}:\n{texto_personalizado}\n")

### 5.1.1
* df.loc[ ]: Este método do pandas é usado para selecionar dados com base nos rótulos (labels) das linhas e colunas.

* df.index[466:470]: Esta parte seleciona as linhas com base nos índices do DataFrame, especificamente da linha de índice 466 até 469 (o índice 470 é excluído, já que o intervalo 466:470 é exclusivo do último valor). Ou seja, são 4 linhas selecionadas.

* O resultado é que df_selecionado será um novo DataFrame que contém as linhas de índice 466, 467, 468 e 469 do DataFrame original df.

### 5.1.2 O for index, row in df_selecionado.iterrows( ):



### 5.1.3 Objetivo: Iterar pelas linhas do DataFrame df_selecionado.

* df_selecionado.iterrows( ): Retorna um gerador que percorre cada linha do DataFrame. Em cada iteração, o método iterrows( ) retorna um índice (index) e uma série (row) contendo os dados da linha.
Por exemplo, se o DataFrame tem 4 colunas (como nome, cpf, etc.), então row será uma série contendo esses valores para a linha atual.

### 5.1.4 A variavel texto_personalizado = script.format( nome=row['nome'], cpf=row['cpf'], setor=row['setor'], filial=row['filial'], usuario_espelho=row['usuario_espelho'], telefone=row['telefone'] )Objetivo: Criar um texto personalizado para cada linha do DataFrame.





### 5.1.5

* script.format(...): Usa a função format() de strings em Python para substituir os placeholders no template script com os valores reais extraídos da linha row.

* row['nome'], row['cpf'], etc.: Extrai o valor correspondente à coluna especificada da linha atual (a série row).
  row['nome']: Valor da coluna nome na linha atual.
  row['cpf']: Valor da coluna cpf na linha atual.

* Da mesma forma para as outras colunas: setor, filial, usuario_espelho, telefone.

* A função format() substitui esses placeholders ({nome}, {cpf}, etc.) dentro da variável script com os valores reais extraídos da linha, gerando o texto personalizado.


### 5.1.6 print(f"Texto gerado para {row['nome']}:\n{texto_personalizado}\n")



### 5.1.7 Objetivo: Exibir o texto gerado no console.

* (f"Texto gerado para {row['nome']}:\n{texto_personalizado}\n"): A f-string permite a interpolação de variáveis diretamente na string. Aqui, o nome da pessoa (row['nome']) é incluído na mensagem.
* \n: Inserção de quebras de linha para melhorar a legibilidade da saída no console.
* O resultado é que para cada iteração, o texto personalizado gerado será exibido, indicando para qual pessoa ({row['nome']}) o texto foi gerado.

### 5.1.8. indices_especificos = [466, 467, 468, 469]
Objetivo: Definir uma lista de índices de linhas específicas que você deseja processar do DataFrame df.

Detalhe:

Aqui, a variável indices_especificos é uma lista que contém os números de índices das linhas que você quer processar.
No exemplo, a lista contém os índices [466, 467, 468, 469], o que significa que você quer selecionar essas 4 linhas específicas do DataFrame, com base nos seus índices.

## 5.2 Gerar texto pegando por Índice Expecífico



In [ ]:
# Índices das linhas que deseja processar
indices_especificos = [466, 467, 468, 469]  # Exemplo: processar a primeira, terceira e quinta linha

# Selecionar essas linhas
df_selecionado = df.loc[indices_especificos]

# Gerar textos apenas para essas linhas
for index, row in df_selecionado.iterrows():
    texto_personalizado = script.format(
         nome=row['nome'],
        cpf=row['cpf'],
        setor=row['setor'],
        filial=row['filial'],
        usuario_espelho=row['usuario_espelho'],
        telefone=row['telefone']
    )
    print(f"Texto gerado para {row['nome']}:\n{texto_personalizado}\n")


#### 5.2.1.

indices_especificos = [466, 467, 468, 469]

Objetivo: Definir uma lista de índices de linhas específicas que você deseja processar do DataFrame df.

Detalhe:

Aqui, a variável indices_especificos é uma lista que contém os números de índices das linhas que você quer processar.
No exemplo, a lista contém os índices [466, 467, 468, 469], o que significa que você quer selecionar essas 4 linhas específicas do DataFrame, com base nos seus índices.

#### 5.2.2.
df_selecionado = df.loc[indices_especificos]

Objetivo: Selecionar as linhas do DataFrame df com os índices listados em indices_especificos.

Detalhe:

df.loc[]: Este é um método de indexação por rótulos no pandas. Ele é usado para selecionar linhas e colunas com base nos rótulos de índice, não pela posição numérica da linha.

Uma outra opção de seleção seria:

Se você quiser selecionar linhas pela posição (em vez de rótulos de índice), usaria df.iloc[].

df.loc[indices_especificos]: Acessa as linhas cujos índices são exatamente os listados em indices_especificos. Ou seja, seleciona as linhas com índice 466, 467, 468, e 469 no DataFrame df.

Resultado:

O resultado dessa linha é um novo DataFrame df_selecionado que contém apenas as 4 linhas de df cujos índices são 466, 467, 468 e 469.